In [4]:
from annoy import AnnoyIndex
import numpy as np
import torch
from tqdm import tqdm_notebook
from argparse import Namespace

In [5]:
args = Namespace(
    glove_filename='data/glove.6B.100d.txt'
)

In [15]:
def load_word_vectors(filename):
    word_to_index = {}
    word_vectors = []
    
    with open(filename) as fp:
        for line in tqdm_notebook(fp.readlines(), leave=False):
            line = line.split(" ")
            
            word = line[0]
            word_to_index[word] = len(word_to_index)
            
            vec = np.array([float(x) for x in line[1:]])
            word_vectors.append(vec)
            
    return word_to_index, word_vectors

In [16]:
class PreTrainedEmbeddings(object):
    def __init__(self, glove_filename):
        self.word_to_index, self.word_vectors = load_word_vectors(glove_filename)
        self.word_vector_size = len(self.word_vectors[0])
        
        self.index_to_word = {v: k for k, v in self.word_to_index.items()}
        self.index = AnnoyIndex(self.word_vector_size, metric='euclidean')
        print('Building Index')
        for _, i in tqdm_notebook(self.word_to_index.items(), leave=False):
            self.index.add_item(i, self.word_vectors[i])
        self.index.build(50)
        print('Finished!')
    
    def get_embedding(self, word):
        return self.word_vectors[self.word_to_index[word]]
    
    def closest(self, word, n=1):
        vector = self.get_embedding(word)
        nn_indices = self.index.get_nns_by_vector(vector, n)
        return [self.index_to_word[neighbor] for neighbor in nn_indices]
    
    def closest_v(self, vector, n=1):
        nn_indices = self.index.get_nns_by_vector(vector, n)
        return [self.index_to_word[neighbor] for neighbor in nn_indices]
    
    def sim(self, w1, w2):
        return np.dot(self.get_embedding(w1), self.get_embedding(w2))

In [17]:
glove = PreTrainedEmbeddings(args.glove_filename)

Building Index


Finished!


In [18]:
glove.closest('apple', n=5)

['apple', 'microsoft', 'dell', 'pc', 'compaq']

In [19]:
glove.closest('plane', n=5)

['plane', 'airplane', 'jet', 'flight', 'crashed']

In [34]:
glove.sim('beer', 'wine'), glove.sim('beer', 'gasoline')

(26.873448266652, 16.501491855324)

** Lexical relationships uncovered by word embeddings **

In [35]:
def SAT_analogy(w1, w2, w3):
    '''
    Solves problems of the type:
    w1 : w2 :: w3 : __
    '''
    closest_words = []
    try:
        w1v = glove.get_embedding(w1)
        w2v = glove.get_embedding(w2)
        w3v = glove.get_embedding(w3)
        w4v = w3v + (w2v - w1v)
        closest_words = glove.closest_v(w4v, n=5)
        closest_words = [w for w in closest_words if w not in [w1, w2, w3]]
    except:
        pass
    if len(closest_words) == 0:
        print(':-(')
    else:
        print('{} : {} :: {} : {}'.format(w1, w2, w3, closest_words[0]))

**Pronouns**

In [36]:
SAT_analogy('man', 'he', 'woman')

man : he :: woman : she


** Verb-Noun relationships **

In [37]:
SAT_analogy('fly', 'plane', 'sail')

fly : plane :: sail : ship


**Noun-Noun relationships**

In [38]:
SAT_analogy('cat', 'kitten', 'dog')

cat : kitten :: dog : pug


**Hypernymy**

In [39]:
SAT_analogy('blue', 'color', 'dog')

blue : color :: dog : animal


**Meronymy**

In [40]:
SAT_analogy('leg', 'legs', 'hand')

leg : legs :: hand : hands


**Troponymy**

In [41]:
SAT_analogy('talk', 'communicate', 'read')

talk : communicate :: read : correctly


**Metonymy**

In [42]:
SAT_analogy('blue', 'democrat', 'red')

blue : democrat :: red : republican


**Misc**

In [43]:
SAT_analogy('man', 'doctor', 'woman')

man : doctor :: woman : nurse


In [44]:
SAT_analogy('man', 'leader', 'woman')

man : leader :: woman : opposition
